In [1]:
# 상위 폴더 경로 가져오기.
from example_base import GetParentPath
# 핑퐁 로봇 제어 모듈 import.
from pingpongthread import PingPongThread

In [2]:
# 2개 로봇을 연결하는 인스턴스 생성.
PingPongThreadInstance = PingPongThread(number=2, group_id=4)
# 로봇 제어 쓰레드 시작.
PingPongThreadInstance.start()
# 모든 로봇이 연결될 때 까지 기다림.
PingPongThreadInstance.wait_until_full_connect()

Found device: nRF Connect USB CDC ACM(COM12)
Serial connected.
Write data: FF FF 04 FF 20 00 AD 00 0B 1A 04
Connected with a master robot.
Connected robots: 2
Fully connected.


In [3]:
# '우리나라 지폐' 사진 클래스 인스턴스, '위조 지폐' 클래스 인스턴스 생성.
bill = PingPongThreadInstance.ImageClass("bill", "chapter6/bill")
fake = PingPongThreadInstance.ImageClass("fake", "chapter6/fake")

In [4]:
# 모델 학습. 
# 첫 번째 인수는 저장하는 이름, 두 번째 인수는 knn 알고리즘의 k 값, 세 번째 인수는 모델 모드. 네 번째 인수 이후는 클래스 인스턴스들.
model = PingPongThreadInstance.train_classes("chapter6/bill_model.json", 1, 2, bill, fake)

Class bill : 8 images.
Class fake : 8 images.
Training done.
chapter6/bill_model.json saved.


In [5]:
# 시간 제어를 위한 모듈 import.
import time
# 키보드 제어를 위한 모듈 import.
import keyboard

# 대사 출력.
print("여기에 지폐를 인식시키면")
time.sleep(1)
print("위조지폐 여부를 알 수 있대")
time.sleep(1)
# '스페이스 바' 키를 누르면 진행.
while True:
    if keyboard.is_pressed(" "):
        break
    else:
        time.sleep(0.01)
time.sleep(4)
print("재미있겠다~ 빨리 해보자!")
time.sleep(2)

여기에 지폐를 인식시키면
위조지폐 여부를 알 수 있대
재미있겠다~ 빨리 해보자!


In [ ]:
# 대사 읽어주기.
PingPongThreadInstance.tts_ko("지폐를 인식시켜 주세요", True)
time.sleep(2)

In [6]:
# 웹캠 열기.
# PingPongThreadInstance.webcam_open(0)
PingPongThreadInstance.webcam_open("http://192.168.66.1:9527/videostream.cgi?loginuse=admin&loginpas=admin")

In [7]:
# 프레임을 평가하는 인스턴스 생성. 누적 프레임은 3초 동안 보관.
frames_predictor = PingPongThreadInstance.FramesPredictor(model=model, timer_sec=3)
# 웹캠을 이용하여 지폐를 예측하는 루프.
while True:
    # 주피터 노트북 출력 비우기.
    PingPongThreadInstance.clear_output()
    # 현재 웹캠 프레임을 보여주고 가져오기.
    frame = PingPongThreadInstance.webcam_get_frame(window="Get_frame")
    # 현재 프레임을 평가하고, 평가 내용을 누적.
    frame_prediction = frames_predictor.image_predict_and_accum(frame)
    print(frame_prediction)
    # 누적된 평가 내용.
    accum_prediction = frames_predictor.accum_predict()
    # 누적된 평가 내용이 없으면 돌아가기. 아니면 가장 확률이 높은 클래스 확인.
    if accum_prediction == None:
        max_class = None
        continue
    else:
        print("accum_prediction:", accum_prediction)
        # 가장 확률이 높은 클래스.
        max_class = max(accum_prediction, key=accum_prediction.get)
    # '우리나라 지폐'일 확률이 90% 이상일 때.
    if max_class == "bill" and accum_prediction[max_class] > 0.9:
        # 클래스 읽어주기.
        PingPongThreadInstance.tts_ko("우리나라 지폐", True)
        # 50도만큼 1번 큐브를 반시계 방향으로 회전, 2번 큐브를 시계 방향으로 회전.
        angle = 50/360
        PingPongThreadInstance.run_motor_step([1, 2], 15, [-angle, angle])
        time.sleep(angle/15*60)
        # 50도만큼 1번 큐브를 시계 방향으로 회전, 2번 큐브를 반시계 방향으로 회전.
        PingPongThreadInstance.run_motor_step([1, 2], 15, [angle, -angle])
        time.sleep(angle/15*60)
        break
    # '위조 지폐'일 확률이 90% 이상일 때.
    elif max_class == "fake" and accum_prediction[max_class] > 0.9:
        # 반박자로 '미, 도, 미, 도' 연주.
        PingPongThreadInstance.play_music(1, ["mi", "do", "mi", "do"], ["half"]*4)
        # 클래스 읽어주기.
        PingPongThreadInstance.tts_ko("위조 지폐", True)
        PingPongThreadInstance.tts_ko("위조 지폐", True)
        # 45도만큼 1번 큐브를 시계 방향으로 회전, 2번 큐브를 시계 방향으로 회전.
        angle = 45/360
        PingPongThreadInstance.run_motor_step([1, 2], 15, [angle, angle])
        time.sleep(angle/15*60)
        # 45도만큼 1번 큐브를 반시계 방향으로 회전, 2번 큐브를 반시계 방향으로 회전.
        PingPongThreadInstance.run_motor_step([1, 2], 15, [-angle, -angle])
        time.sleep(angle/15*60)
        # 45도만큼 1번 큐브를 반시계 방향으로 회전, 2번 큐브를 반시계 방향으로 회전.
        PingPongThreadInstance.run_motor_step([1, 2], 15, [-angle, -angle])
        time.sleep(angle/15*60)
        # 45도만큼 1번 큐브를 시계 방향으로 회전, 2번 큐브를 시계 방향으로 회전.
        PingPongThreadInstance.run_motor_step([1, 2], 15, [angle, angle])
        time.sleep(angle/15*60)
        break
    # 모두 아니면 돌아가기.
    else:
        continue

Frame is False
{'bill': 0, 'fake': 1}
accum_prediction: {'fake': 1.0}
Write data: FF FF 04 00 00 A1 E8 00 17 00 00 2C 3C 00 28 3C 00 2C 3C 00 28 3C 00
위조 지폐
위조 지폐
Write data: FF FF 04 AA 20 00 CD 00 33 02 01 00 00 FF FF 04 00 20 00 C1 00 13 02 01 00 02 03 84 00 00 00 FA FF FF 04 01 20 00 C1 00 13 02 01 00 02 03 84 00 00 00 FA
Timeout. Initialize buffer.
Timeout buffer: 00 52 04 00 00 A1 E8 00 17 00 00
Aggregator set.
Write data: FF FF 04 AA 20 00 CD 00 33 02 01 00 00 FF FF 04 00 20 00 C1 00 13 02 01 00 02 FC 7C 00 00 00 FA FF FF 04 01 20 00 C1 00 13 02 01 00 02 FC 7C 00 00 00 FA
Aggregator set.
Write data: FF FF 04 AA 20 00 CD 00 33 02 01 00 00 FF FF 04 00 20 00 C1 00 13 02 01 00 02 FC 7C 00 00 00 FA FF FF 04 01 20 00 C1 00 13 02 01 00 02 FC 7C 00 00 00 FA
Aggregator set.
Write data: FF FF 04 AA 20 00 CD 00 33 02 01 00 00 FF FF 04 00 20 00 C1 00 13 02 01 00 02 03 84 00 00 00 FA FF FF 04 01 20 00 C1 00 13 02 01 00 02 03 84 00 00 00 FA
Aggregator set.


In [8]:
# 웹캠 닫기.
PingPongThreadInstance.webcam_close()
# 로봇 제어 쓰레드 종료.
PingPongThreadInstance.end()

Write data: FF FF FF FF 00 00 A8 00 0A 01
Disconnect master robot.
Serial disconnected. Sleep 3 seconds.
End thread.
